## Ridge Regression with Gradient Descent

We will build a Linear Regression model with Ridge regularization to balance fit and magnitude of coefficients using Gradient Descent optimization for learning purpose.

**Cost Function**: 
$$ J(w) = \frac{1}{2m} [\sum_{i=1}^m (y_{i} - h(x_{i})^{T}w)^{2} + \lambda\sum_{j=1}^nw^{2}_{j}]$$  
**Compute the Derivative**: 
$$ w_{j} := w_{j} - \alpha[\frac{1}{m} (\sum_{i=1}^mh_{j}(x_{j})(y_{i} - h(x_{i})^{T}w^{(t)})) + \frac{\lambda}{m}w_{j}] $$  



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_csv('/Users/weijunchen/Py_project/MLfromScratch/Machine-Learning-from-Scratch/Multi_linear.csv')

In [3]:
df.columns = ['x1', 'x2', 'y']

Let's use a simple data set for this demo.
(data [source](https://www.coursera.org/learn/machine-learning))


In [4]:
df.head(5)

,x1,x2,y
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


Add some polynomial terms

In [5]:
df['x3'] = df['x2'].values**2
df['x4'] = df['x2'].values**3
df['x5'] = df['x2'].values**4
df['x6'] = df['x2'].values**5
df['x7'] = df['x1'].values**2

#### Let's define functions to initialize weights, calculate cost function and derivative for GD.

In [6]:
def featureNormalization(X):
    mean=np.mean(X,axis=0)
    std=np.std(X,axis=0)
    X_norm = (X - mean)/std
    return X_norm

In [7]:
def initialize_weights(dimention):
    """
    This function creates a vector of shape (dim, 1) for parameter w
    
    Argument:
    dimention -- number of parameters
    """
    w = np.ones(shape=(dimention, 1))
    assert(w.shape == (dimention, 1))
    return w

In [8]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [9]:
def feature_derivative_ridge(errors, feature, weight, l2_penalty, feature_is_constant):
    if feature_is_constant == True:
        derivative = 2 * np.dot(errors.T, feature)
    else:
        derivative = 2 * np.dot(errors.T, feature) + 2*(l2_penalty*weight)
    return derivative

In [10]:
def cost(feature_matrix, output, weights, l2_penalty):
    m=len(output)
    predictions = predict_output(feature_matrix, weights)
    square_error = (predictions - y)**2
    # not include intercept term
    l2_term = weights[1:]**2
    return 1/(2*m) * (np.sum(square_error) + np.sum(l2_term))

In [11]:
def ridge_regression_gradient_descent(feature_matrix, output, initial_weights, step_size, l2_penalty, max_iterations=100):
    weights = np.array(initial_weights) # make sure it's a numpy array
    
    cost_history = []
    while max_iterations > 0:
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output
        for i in range(len(weights)): 
            if i == 0:
                feature_is_constant = True
            else:
                feature_is_constant = False
            # calculate derivative
            derivative = feature_derivative_ridge(errors, feature_matrix[:,i], weights[i], l2_penalty, feature_is_constant)
            weights[i] = weights[i] - (step_size * derivative)
        cost_history.append(cost(feature_matrix, output, weights, l2_penalty))
        max_iterations -= 1            
    return weights, cost_history


#### Prepare data and normalize it

In [12]:
m = df.shape[0]
y = df['y'].values.reshape(m,1)
X = df.drop(columns=['y']).values
X = featureNormalization(X)
X = np.append(np.ones((m,1)),X,axis=1)

#### Update weights using gradient descent

In [13]:
n = X.shape[1]
initial_weights = initialize_weights(n)
step_size = 1e-8
max_iterations = 500

no_l2_penalty = 0
l2_penalty = 1e6

#### No L2 penalty: $\lambda = 0 $

In [14]:
w1, cost_history1 = ridge_regression_gradient_descent(X, y, 
                                                               initial_weights, step_size, 
                                                               no_l2_penalty, max_iterations)

In [15]:
print("h(x) = "+str(round(w1[0,0],2))+" + "\
      +str(round(w1[1,0],2))+"x1 + " \
      +str(round(w1[2,0],2))+"x2 + " \
      +str(round(w1[3,0],2))+"x3 + " \
      +str(round(w1[4,0],2))+"x4 + " \
      +str(round(w1[5,0],2))+"x5 + " \
      +str(round(w1[6,0],2))+"x6 + " \
      +str(round(w1[7,0],2))+"x7")

h(x) = 160.96 + 50.66x1 + 26.67x2 + 27.68x3 + 28.44x4 + 29.07x5 + 29.55x6 + 49.4x7


#### with L2 penalty: $\lambda > 0 $

In [16]:
w2, cost_history2 = ridge_regression_gradient_descent(X, y, 
                                                      initial_weights, step_size, 
                                                      l2_penalty, max_iterations)

In [17]:
print("h(x) = "+str(round(w2[0,0],2))+" + "\
      +str(round(w2[1,0],2))+"x1 + " \
      +str(round(w2[2,0],2))+"x2 + " \
      +str(round(w2[3,0],2))+"x3 + " \
      +str(round(w2[4,0],2))+"x4 + " \
      +str(round(w2[5,0],2))+"x5 + " \
      +str(round(w2[6,0],2))+"x6 + " \
      +str(round(w2[7,0],2))+"x7")

h(x) = 160.96 + 4.97x1 + 2.57x2 + 2.67x3 + 2.75x4 + 2.81x5 + 2.86x6 + 4.84x7
